# Main notebook - experimenting with guesses

SPDX-License-Identifier: 0BSD

This does semantic search based solely on repository name and user input.

Compare to
[`findrepo`](https://github.com/EliahKagan/newrepo-findrepo#using-findrepo).

In [1]:
import logging

import numpy as np
from tabulate import tabulate

import fr2ex

In [2]:
logging.basicConfig(level=logging.INFO)

In [3]:
names = fr2ex.remote.fetch_repo_names()

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.6p1)
INFO:paramiko.transport:Authentication (publickey) successful!
INFO:paramiko.transport.sftp:[chan 0] Opened sftp connection (server version 3)
INFO:paramiko.transport.sftp:[chan 0] sftp session closed.


In [4]:
len(names)

698

In [5]:
moderation = fr2ex.moderation.get_moderation(names)

INFO:root:Querying OpenAI moderation endpoint.


In [6]:
len(moderation)

698

In [7]:
any(result['flagged'] for result in moderation)

False

In [8]:
fr2ex.tokens.report_cost(names)

It looks like the rate is $0.0004 per 1000 tokens. If so, the cost to
process 1984 tokens is about $0.0007936 (that is, 0.0793600 cents).


In [9]:
embeddings = fr2ex.embedding.embed_many(names)

INFO:root:Querying OpenAI embeddings endpoint.


In [10]:
def guess(name: str, count: int = 5) -> None:
    """Show top guesses for similarity of name to already embedded names."""
    scores = embeddings @ fr2ex.embedding.embed(name)
    ordering = sorted(zip(scores, names), reverse=True)
    table = [(name, score) for score, name in ordering]
    return tabulate(table[:count], tablefmt='html', floatfmt='.6f')

In [11]:
guess('algorithm')

Calculator,0.868787
Geometry,0.865322
algorithms-suggestions,0.860121
Alias,0.851815
crystal-algo,0.849683


In [12]:
guess('sorting')

SortingRanges,0.932616
Sorts,0.909054
sortkey,0.859548
PartialSort,0.855047
tsort,0.840450


In [13]:
guess('algorithm-visualization')

algorithms-suggestions,0.889051
algorithms-python,0.859356
crystal-algo,0.856027
Graph-gist,0.842658
codegraph,0.833195


In [14]:
guess('maven')

try-maven,0.915951
mavener,0.879947
retry-maven,0.868217
pimpl,0.836310
try-gradle,0.833699


In [15]:
guess('find-repo')

newrepo-findrepo,0.916233
findrepo2-experiment,0.867456
backup-repos,0.865144
find-articles,0.857105
TestRepo,0.842453


In [16]:
guess('graph theory', count=10)

codegraph,0.881251
Graph,0.879454
Graph-gist,0.864824
Geometry,0.839709
wpf-graph,0.829149
graphyaml,0.820673
gh-profile,0.814760
c-sketches,0.813890
trig,0.809229
Draw,0.808864


In [17]:
guess('shortest paths', count=10)

Dijkstra,0.891423
Paths,0.838847
TreeTraversal,0.830639
Kruskal,0.816365
algorithms-suggestions,0.816244
Traverse,0.812973
Permutations,0.806550
treejs,0.801372
DFS,0.800755
kt-short,0.800324


In [18]:
guess('shortest-paths', count=10)

Dijkstra,0.892498
Paths,0.836553
TreeTraversal,0.836224
Traverse,0.819850
algorithms-suggestions,0.819177
Kruskal,0.813978
type-arrows,0.812921
kt-short,0.811127
sequence-dag,0.809256
dot-and-cross,0.806420


In [19]:
guess('graph traversal', count=10)

TreeTraversal,0.897430
Traverse,0.869958
Graph,0.868274
codegraph,0.856913
Graph-gist,0.851086
TraverseTree,0.850041
TreeTraversalAnimations,0.844913
Dijkstra,0.825562
DFS,0.823535
sequence-dag,0.819539


In [20]:
guess('graph-traversal', count=10)

TreeTraversal,0.904181
Traverse,0.879860
Graph,0.878266
Graph-gist,0.875246
codegraph,0.872212
TraverseTree,0.865436
TreeTraversalAnimations,0.851144
Dijkstra,0.842906
wpf-graph,0.840953
sequence-dag,0.837051


In [21]:
guess('graphics', count=10)

Graph,0.889289
printing,0.882984
Geometry,0.881471
Draw,0.875394
fps,0.859528
works,0.848890
coordinates,0.845146
Components,0.842697
interop,0.841286
Benchmark,0.840062


In [22]:
guess('deluge')

Flood,0.865072
foobar,0.825893
lambda-demo,0.819155
Dynamo,0.816880
yule,0.813996


In [23]:
guess('source control')

git,0.823033
Linkage,0.807321
gitscripts,0.806840
capture.net,0.803207
test-site,0.801885


In [24]:
guess('source-control')

git,0.831532
git-notes,0.830648
dot-imports,0.815999
gitscripts,0.813048
editor-configs,0.812764
